In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from src.fluxonium import *

In [2]:
N = 136
CJb = 5.73 #fF
CJ = 29 #fF
C0_jja = 57e-3 
C0 = C0_jja * np.ones(N + 1)
C0[0] = C0[-1] = 1.473
C0[30] = 1.44
C0[60] = 1.648
C0[76] = 1.648
C0[106] = 1.44
generate_C_x_1(N, CJ, C0, CJb)[0,0]

7.538509731833933